In [16]:
import sys
import numpy as np
import copy
from functools import lru_cache,wraps
from operator import itemgetter,countOf
import matplotlib.pyplot as plt

import collections

def read_file(file_name):
    with open(file_name) as input_file:
        list_of_lines=input_file.read().splitlines()
    i=0;
    operations=[]
    for line in list_of_lines:
        List=line.split(' ')
        
        if 'on' in List[0]:
            op=1
        else:
            op=0
        List[1]=List[1].replace('x=','')
        List[1]=List[1].replace('y=','')
        List[1]=List[1].replace('z=','')
        List[1]=List[1].split(',')
        X=List[1][0].split('..')
        Y=List[1][1].split('..')
        Z=List[1][2].split('..')
        operations.append([op,int(X[0]),int(X[1]),int(Y[0]),int(Y[1]),int(Z[0]),int(Z[1])])
    return operations
    
    
if __name__ == '__main__':
        
    Operations = read_file('input-test')
    Cube=np.zeros((101,101,101),np.int64)
    #print( Operations)
    for line in Operations:
        n=line[0]
        for x in range(max([-50,line[1]]),min([51,line[2]+1])):
            for y in range(max([-50,line[3]]),min([51,line[4]+1])):
                for z in range(max([-50,line[5]]),min([51,line[6]+1])):
                    Cube[x+50][y+50][z+50]=1 if n==1 else 0
    print(np.sum(Cube))
    x = range(3, 1)
    for n in x:
        print(n) 

590784


In [1]:
import sys
import numpy as np
import copy
from functools import lru_cache,wraps
from operator import itemgetter,countOf
import matplotlib.pyplot as plt

import collections

def read_file(file_name):
    with open(file_name) as input_file:
        list_of_lines=input_file.read().splitlines()
    i=0;
    operations=[]
    for line in list_of_lines:
        List=line.split(' ')
        
        if 'on' in List[0]:
            op=1
        else:
            op=0
        List[1]=List[1].replace('x=','')
        List[1]=List[1].replace('y=','')
        List[1]=List[1].replace('z=','')
        List[1]=List[1].split(',')
        X=List[1][0].split('..')
        Y=List[1][1].split('..')
        Z=List[1][2].split('..')
        operations.append([op,int(X[0]),int(X[1]),int(Y[0]),int(Y[1]),int(Z[0]),int(Z[1])])
    return operations
    
def Intersect(j,i):
    xm1,xM1,ym1,yM1,zm1,zM1=j[1:7]
    xm2,xM2,ym2,yM2,zm2,zM2=i[1:7]
    cube=[]
    #If there is an intersection
    if ( (xM2-xm1)*(xM1-xm2)>0 and
         (yM2-ym1)*(yM1-ym2)>0 and
         (zM2-zm1)*(zM1-zm2)>0 ):
        xm,xM,ym,yM,zm,zM=Get_coordinates_intersection(xm1,xM1,ym1,yM1,zm1,zM1,xm2,xM2,ym2,yM2,zm2,zM2)
        cube.append((xM-xm+1)*(yM-ym+1)*(zM-zm+1))
        cube.append(xm)
        cube.append(xM)
        cube.append(ym)
        cube.append(yM)
        cube.append(zm)
        cube.append(zM)
        return cube,True
    else:
        return cube,False

def Get_coordinates_intersection(xm1,xM1,ym1,yM1,zm1,zM1,xm2,xM2,ym2,yM2,zm2,zM2):
    xM=min([xM1,xM2])
    xm=max([xm1,xm2])
    yM=min([yM1,yM2])
    ym=max([ym1,ym2])
    zM=min([zM1,zM2])
    zm=max([zm1,zm2])
    return xm,xM,ym,yM,zm,zM

def Size_of_cube(Operations,rank):
    return abs( (Operations[rank][2]-Operations[rank][1]+1)
              * (Operations[rank][4]-Operations[rank][3]+1)
              * (Operations[rank][6]-Operations[rank][5]+1)
              )
    
# This function has found an "on" step at position rank, 
# followed only by off steps.
def Propagate(Operations,rank,N):
    #number of cubes turned on in the first step
    Nb=Size_of_cube(Operations,rank)
    if rank==N-1:
        return Nb
    #if negative steps follow
    else:
        #remove all intersecting negative cubes and compute their intersections two by two
        #them and make a list of resulting cubes
        List=[]
        print('cube on {}'.format(Operations[rank]))
        #go through each following off filter
        for i in range(rank+1,N):
            #is there an intersection?
            cube,Bool=Intersect(Operations[rank],Operations[i])
            if Bool:
                print('Filter off applies')
                print(Operations[i])
                print('overlap')
                print(cube)
                #if the region is new, subtract the new cubebs and add it in List
                if cube not in List:
                    Nb -= New_Cubes(cube,List)
                    List.append(cube)
    return Nb
#res = tuple(tuple(sub) for sub in test_list)

def New_Cubes(cube,List):
    nb=cube[0]
#@lru_cache(maxsize=1000)
def Iterate(List,Effect):
    DeltaN=0
    for cube in List:
        DeltaN += cube[0]*Effect
    #If there is more than one cube
    if(len(List)>1):
        List_Intersection=[]
        for i in range(0,len(List)-1):
            for j in range(i+1,len(List)):
                cube,Bool=Intersect(List[i],List[j])
                if Bool:
                    #print('Intersection')
                    #print(List[i])
                    #print(List[j])
                    print(cube)
                    List_Intersection.append(cube)
        #if List non empty, continue
        if List_Intersection:  
            #print(len(List_Intersection))
            #print(List_Intersection)
            #we iterate with the interactions of the remaining cubes and an opposed effect
            DeltaN+=Iterate(List_Intersection,-Effect)
    return DeltaN

if __name__ == '__main__':
    
    Operations = read_file('input-test')
    N=len(Operations)
    Nb=0;
    #go through the list from the end.
    for rank in range(N-1,-1,-1):
        print('rank={} Nb={}'.format(rank,Nb))
        #if you meet an "on" step, propagate it to the end 
        #and transform it into a negative step to avoid double-counting
        if Operations[rank][0]==1:
            Nb+=Propagate(Operations,rank,N)
            Operations[rank][0]=0
    print(Nb)
    print(2758514936282235)

rank=21 Nb=0
rank=20 Nb=21049844681660
cube on [1, -54112, -39298, -85059, -49293, -27449, 7877]
rank=19 Nb=39769201766995
cube on [1, -41, 9, -7, 43, -33, 15]
rank=18 Nb=39769201894444
rank=17 Nb=39769201894444
cube on [1, -49, -5, -3, 45, -29, 18]
Filter off applies
[0, -41, 9, -7, 43, -33, 15]
overlap
[78255, -41, -5, -3, 43, -29, 15]


TypeError: unsupported operand type(s) for -=: 'int' and 'NoneType'

In [17]:
import sys
import numpy as np
import copy
from functools import lru_cache,wraps
from operator import itemgetter,countOf
import matplotlib.pyplot as plt

from collections import Counter

def read_file(file_name):
    with open(file_name) as input_file:
        list_of_lines=input_file.read().splitlines()
    i=0;
    operations=[]
    for line in list_of_lines:
        List=line.split(' ')
        
        if 'on' in List[0]:
            op=1
        else:
            op=0
        List[1]=List[1].replace('x=','')
        List[1]=List[1].replace('y=','')
        List[1]=List[1].replace('z=','')
        List[1]=List[1].split(',')
        X=List[1][0].split('..')
        Y=List[1][1].split('..')
        Z=List[1][2].split('..')
        operations.append([op,int(X[0]),int(X[1]),int(Y[0]),int(Y[1]),int(Z[0]),int(Z[1])])
    return operations

def Is_In(x,y,z,op):
    return x>=op[1] and x<=op[2] and y>=op[3] and y<=op[4] and z>=op[5] and z<=op[6]

def Rules(x,y,z,Operations):
    n=0
    for op in reversed(Operations):
        if Is_In(x,y,z,op):
            n=op[0]
            return n
    return n
    
if __name__ == '__main__':
    Operations = read_file('input-test')
    X,Y,Z=[],[],[]
    print(Operations)
    print(len(Operations))
    for i in Operations:
        X.extend([i[1],i[2]+1])
        Y.extend([i[3],i[4]+1])
        Z.extend([i[5],i[6]+1])
    X = list(dict.fromkeys(X))
    Y = list(dict.fromkeys(Y))
    Z = list(dict.fromkeys(Z))

    X.sort()
    Y.sort()
    Z.sort()
    print(Counter(X))
    print(Counter(Y))
    print(Counter(Z))
    Nb=0
    for i in range(0,len(X)-1):
        x=int(X[i])
        for j in range(0,len(Y)-1):
            y=int(Y[j])
            for k in range(0,len(Z)-1):
                    z=int(Z[k])
                    
                    #print(x,y,z)
                    value = Rules(x,y,z,Operations)
                    if value==1:
                        Nb += abs((X[i+1]-X[i])*(Y[i+1]-Y[i])*(Z[i+1]-Z[i]))
    
    print(Nb)
    print(2758514936282235)

[[1, -20, 26, -36, 17, -47, 7], [1, -20, 33, -21, 23, -26, 28], [1, -22, 28, -29, 23, -38, 16], [1, -46, 7, -6, 46, -50, -1], [1, -49, 1, -3, 46, -24, 28], [1, 2, 47, -22, 22, -23, 27], [1, -27, 23, -28, 26, -21, 29], [1, -39, 5, -6, 47, -3, 44], [1, -30, 21, -8, 43, -13, 34], [1, -22, 26, -27, 20, -29, 19], [0, -48, -32, 26, 41, -47, -37], [1, -12, 35, 6, 50, -50, -2], [0, -48, -32, -32, -16, -15, -5], [1, -18, 26, -33, 15, -7, 46], [0, -40, -22, -38, -28, 23, 41], [1, -16, 35, -41, 10, -47, 6], [0, -32, -23, 11, 30, -14, 3], [1, -49, -5, -3, 45, -29, 18], [0, 18, 30, -20, -8, -3, 13], [1, -41, 9, -7, 43, -33, 15]]
20
Counter({-49: 1, -48: 1, -46: 1, -41: 1, -40: 1, -39: 1, -32: 1, -31: 1, -30: 1, -27: 1, -22: 1, -21: 1, -20: 1, -18: 1, -16: 1, -12: 1, -4: 1, 2: 1, 6: 1, 8: 1, 10: 1, 18: 1, 22: 1, 24: 1, 27: 1, 29: 1, 31: 1, 34: 1, 36: 1, 48: 1})
Counter({-41: 1, -38: 1, -36: 1, -33: 1, -32: 1, -29: 1, -28: 1, -27: 1, -22: 1, -21: 1, -20: 1, -15: 1, -8: 1, -7: 1, -6: 1, -3: 1, 6: 1, 1